# Predictive Modeling - Apple, Inc. (AAPL)

## Importing Libraries:

In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from datetime import datetime

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

-----

## Company Name

In [114]:
company_name = 'Apple'

## Importing the Data:

In [115]:
def data_reader(company_name):
    company_name=company_name
    df = pd.read_csv(f'data/{company_name}_Clean.csv')
    df['Date'] = pd.to_datetime(df.Date)
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True, ascending=True)
    return df

In [116]:
df = data_reader('Apple')

In [117]:
df.head(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0
1980-12-15,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.402563,0.402563,0.400652,0.400652,43971200.0
1980-12-16,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.373010,0.373010,0.371246,0.371246,26432000.0


### Dropping the Everything Before July 2014:

In [118]:
df.drop(df[:'2014-07-01'].index, inplace=True)

In [119]:
df.head(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2014-07-02,93.865,94.06,93.09,93.480,28465000.0,0.0,1.0,88.348738,88.532278,87.619283,87.986364,28465000.0
2014-07-03,93.670,94.10,93.20,94.030,22891800.0,0.0,1.0,88.165198,88.569928,87.722819,88.504041,22891800.0
2014-07-07,94.140,95.99,94.10,95.968,56468000.0,0.0,1.0,88.607577,90.348856,88.569928,90.328149,56468000.0


-----

# Splitting the Data a Training and Testing Set

## Creating a Function for the Training Set with Feature Engineering:

In [120]:
def shift_dates(df):
    shifted_df = pd.DataFrame(df[:-1].values, index = df[1:].index, columns=df.columns)
    return shifted_df

In [121]:
def lag_and_shift_data(dataframe):
    temp_df = dataframe.copy()
    
    short_SMA = temp_df.rolling(window=12).mean().copy()
    mid_SMA = temp_df.rolling(window=26).mean().copy()
    long_SMA = temp_df.rolling(window=85).mean().copy()

    short_EMA = temp_df.ewm(span=12, adjust=False).mean().copy()
    mid_EMA = temp_df.ewm(span=26, adjust=False).mean().copy()
    long_EMA = temp_df.ewm(span=85, adjust=False).mean().copy()
    
    temp_df = pd.merge(temp_df, short_SMA, left_index=True, right_index=True, suffixes=['','_Short_SMA'])
    temp_df = pd.merge(temp_df, mid_SMA, left_index=True, right_index=True, suffixes=['','_Mid_SMA'])
    temp_df = pd.merge(temp_df, long_SMA, left_index=True, right_index=True, suffixes=['','_Long_SMA'])
    
    temp_df = pd.merge(temp_df, short_EMA, left_index=True, right_index=True, suffixes=['','_Short_EMA'])
    temp_df = pd.merge(temp_df, mid_EMA, left_index=True, right_index=True, suffixes=['','_Mid_EMA'])
    temp_df = pd.merge(temp_df, long_EMA, left_index=True, right_index=True, suffixes=['','_Long_EMA'])
    
    temp_df.dropna(inplace=True) 
    
    temp_df = shift_dates(temp_df)

    return temp_df

In [122]:
df_shift = lag_and_shift_data(df)

In [123]:
df_shift.head()

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-31,106.959,107.35,105.900,106.98,40654793.0,0.0,1.0,101.174038,101.543891,100.172315,...,98.626302,99.449687,5.503428e+07,0.002727,1.0,93.946729,94.716652,93.184824,93.963038,5.503428e+07
2014-11-03,108.010,108.04,107.210,108.00,44639285.0,0.0,1.0,102.168194,102.196572,101.411463,...,98.825923,99.648531,5.479254e+07,0.002664,1.0,94.137926,94.890604,93.376141,94.153636,5.479254e+07
2014-11-04,108.220,110.30,108.010,109.40,52282550.0,0.0,1.0,102.366836,104.334338,102.168194,...,99.039506,99.875310,5.473417e+07,0.002602,1.0,94.329296,95.110226,93.580607,94.370598,5.473417e+07
2014-11-05,109.360,109.49,107.720,108.60,41574365.0,0.0,1.0,103.445178,103.568147,101.893879,...,99.241378,100.078209,5.442812e+07,0.002541,1.0,94.541293,95.306922,93.773939,94.564917,5.442812e+07
2014-11-06,109.100,109.30,108.125,108.86,37435905.0,0.0,1.0,103.199241,103.388423,102.276974,...,99.447974,100.282437,5.403296e+07,0.002482,1.0,94.742641,95.494863,93.971684,94.760435,5.403296e+07


## Taking a Look at the Time-Shifted Data Set:

In [124]:
df_shift.tail(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
2018-03-23,170.00,172.68,168.60,168.845,41051076.0,0.0,1.0,170.00,172.68,168.60,...,169.691723,171.097703,3.358258e+07,0.000518,1.0,171.126419,172.657153,169.671634,171.077463,3.358258e+07
2018-03-26,168.39,169.92,164.94,164.940,40248954.0,0.0,1.0,168.39,169.92,164.94,...,169.581218,170.954500,3.373761e+07,0.000506,1.0,171.062781,172.593499,169.561596,170.934731,3.373761e+07
2018-03-27,168.07,173.10,166.44,172.770,36272617.0,0.0,1.0,168.07,173.10,166.44,...,169.508166,170.996721,3.379657e+07,0.000494,1.0,170.993182,172.605278,169.489000,170.977412,3.379657e+07


In [125]:
# df_shift['2014-07-01':]

## Dropping Everything Before July 2014:

In [126]:
# df_shift.drop(df_shift[:'2014-07-01'].index, inplace=True)

In [127]:
# df_shift.head(3)

In [128]:
def data_split(dataframe):
    temp_df = dataframe.copy()
    X_test = temp_df['2016-07-02':'2017-03-27']
    temp_df.drop(temp_df['2016-03-02':'2016-12-01'].index, inplace=True)
    X_train = temp_df
    return X_train, X_test

In [129]:
X_train, X_test = data_split(df_shift)

## Creating a Function for a Training a Testing Set: 

In [104]:
# def data_split(dataframe):
#     temp_df = dataframe.copy()
#     X_test = temp_df['2016-07-02':'2017-03-27']
#     temp_df.drop(temp_df['2016-07-02':'2017-03-27'].index, inplace=True)
#     X_train = temp_df
#     return X_train, X_test

In [105]:
# X_train, X_test = data_split(df_shift)

### Inspecting the Training Data:

In [130]:
X_train.head(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-31,106.959,107.35,105.90,106.98,40654793.0,0.0,1.0,101.174038,101.543891,100.172315,...,98.626302,99.449687,5.503428e+07,0.002727,1.0,93.946729,94.716652,93.184824,93.963038,5.503428e+07
2014-11-03,108.010,108.04,107.21,108.00,44639285.0,0.0,1.0,102.168194,102.196572,101.411463,...,98.825923,99.648531,5.479254e+07,0.002664,1.0,94.137926,94.890604,93.376141,94.153636,5.479254e+07
2014-11-04,108.220,110.30,108.01,109.40,52282550.0,0.0,1.0,102.366836,104.334338,102.168194,...,99.039506,99.875310,5.473417e+07,0.002602,1.0,94.329296,95.110226,93.580607,94.370598,5.473417e+07


### Inspecting the Testing Data:

In [131]:
X_test.head(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
2016-07-05,95.49,96.465,95.33,95.89,26026540.0,0.0,1.0,93.321192,94.274047,93.164826,...,98.298623,99.153689,3.984747e+07,0.006571,1.0,96.547814,97.431902,95.771692,96.604325,3.984747e+07
2016-07-06,95.39,95.400,94.46,94.99,27705210.0,0.0,1.0,93.223463,93.233236,92.314586,...,98.209353,99.056859,3.956509e+07,0.006418,1.0,96.470504,97.334258,95.691294,96.516610,3.956509e+07
2016-07-07,94.60,95.660,94.37,95.53,30949090.0,0.0,1.0,92.451406,93.487331,92.226630,...,98.120066,98.974839,3.936472e+07,0.006269,1.0,96.377036,97.244795,95.610720,96.443207,3.936472e+07


-----

# Normalizing the Data with a MinMaxScaler

## Instantiating the Scaler:

In [132]:
scaler = MinMaxScaler(feature_range=(-1, 1))

## Scaling the Training Set:

In [133]:
X_train_sc = scaler.fit_transform(X_train.values)

## Scaling the Testing Set:

In [134]:
X_test_sc = scaler.transform(X_test.values)

## Setting the y Training Set:

In [141]:
X_train.shape

(662, 84)

In [142]:
y_train.shape

(854,)

In [135]:
y_train = df[X_train.index[0]:X_train.index[-1]].Close.values

## Setting the y Testing Set

In [136]:
y_test = df[X_test.index[0]:X_test.index[-1]].Close.values

-----

# Random Forest Regression Model

## Setting up the Random Forest (RF) Regressor:

In [137]:
rf = RandomForestRegressor(n_estimators=100, max_depth=15, 
                           min_samples_leaf=3, bootstrap=False, 
                           n_jobs=2, random_state=42)

In [138]:
df.shape

(939, 12)

In [139]:
X_train.shape

(662, 84)

### Fitting the Scaled Data with the RF Model:

In [140]:
rf.fit(X_train_sc, y_train)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/adamdelreal/anaconda3/envs/tensorflow/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/

### Scoring on the Training Data:

In [21]:
rf.score(X_train_sc, y_train)

0.9988511480564479

### Scoring on the Testing Data

In [22]:
rf.score(X_test_sc, y_test)

-3.6526599373079107

### Inspecting the Average Prediction:

In [23]:
y_test.mean()

154.62082337662338

In [24]:
rf.predict(X_test_sc).mean()

182.12251210445518

In [25]:
rf.feature_importances_.mean()

0.011904761904761902

In [26]:
# coef_weights = pd.DataFrame(rf.coef_, index=X_train.columns, columns=['weight'])
# coef_weights.sort_values('weight').tail()

-----

## Decomposing Signal Components with Principal Component Analysis (PCA):

### Instantiating the PCA Decomposition:

In [27]:
pca = PCA(n_components=1, random_state=42)

### Fitting and Transforming the Scaled Training Set with PCA Weights:

In [28]:
X_train_pca = pca.fit_transform(X_train_sc)

### Transforming the Scaled Test Set with PCA Weights:

In [29]:
X_test_pca = pca.transform(X_train_sc)

### Fitting the PCA Weighted Training Data on a Random Forest Regressor Model:

In [30]:
rf.fit(X_train_pca, y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

### Scoring the PCA Weighted Training Data on a Random Forest Regressor Model:

In [31]:
rf.score(X_train_pca, y_train)

0.8824991349411133

### Scoring the PCA Weighted Test Data on a Random Forest Regressor Model:

In [33]:
# rf.score(X_test_sc, y_test)

### Inspecting the Average Prediction Value of the PCA Weighted Training Data on a Random Forest Regressor Model:

In [34]:
rf.predict(X_test_pca).mean()

100.05874912845564

-----

# Grid Searching a Random Forest Regression Model:

In [39]:
from sklearn.model_selection import GridSearchCV

### Creating a Pipeline

In [40]:
pipe = Pipeline([
    ('rf', RandomForestRegressor())
])

### Setting up the Parameters:

In [ ]:
# Number of trees in random forest

n_estimators = [x for x in range(8, 14, 2)]

# Number of features to consider at every split
max_features = ['auto', 'log2']

# Maximum number of levels in tree
max_depth = [x for x in range(1, 3)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [x for x in range(1, 3)]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
params = {'rf__n_estimators': n_estimators,
           'rf__max_features': max_features,
           'rf__max_depth': max_depth,
           'rf__min_samples_split': min_samples_split,
           'rf__min_samples_leaf': min_samples_leaf,
           'rf__bootstrap': bootstrap}
print(params)

### Gridsearching the Parameters:

In [ ]:
rf_search = GridSearchCV(pipe, params, n_jobs=3)

### Fitting the Scaled Data with the Model:

In [ ]:
rf_search.fit(X_train_sc, y_train)

### Scoring the Training Data:

In [ ]:
rf_search.score(X_train_sc, y_train)

### Scoring the Test Data

In [ ]:
rf_search.score(X_test_sc, y_test)

In [ ]:
plt.scatter(y_test, rf_search.predict(X_test_sc))
plt.xlabel('Actual')
plt.ylabel('Predicted')

In [ ]:
coef_weights = pd.DataFrame(rf_search.coef_, index=X_train.columns, columns=['weight'])
coef_weights.sort_values('weight').tail()

- `'rf__n_estimators': [40, 55, 70, 85, 100],` => `n_estimators=10, `


- `'rf__max_features': ['auto', 'sqrt'],` => `max_features='auto',`


- `'rf__max_depth': [2, 80, None],` => -`max_depth=None,`


- `'rf__min_samples_split': [2, 5, 10],` => `min_samples_split=2,`


- `'rf__min_samples_leaf': [1, 2, 4],` => `min_samples_leaf=1,`


- `'rf__bootstrap': [True, False]` => `bootstrap=True,`

--------

## GridSearching a Random Forest with PCA Decomposition:

In [ ]:
pipe_w = Pipeline([
    ('pca', PCA()),
    ('rf', RandomForestRegressor())
])

### Setting the Params

In [ ]:
# Number of trees in random forest
n_estimators_w = [x for x in range(8, 14, 2)]

# Number of features to consider at every split
max_features_w = ['auto', 'log2']

# Maximum number of levels in tree
max_depth_w = [x for x in range(1, 3)]
max_depth_w.append(None)

# Minimum number of samples required to split a node
min_samples_split_w = [x for x in range(1, 5)]

# Minimum number of samples required at each leaf node
min_samples_leaf_w = [x for x in range(1, 3)]

# Method of selecting samples for training each tree
bootstrap_w = [True, False]

# pca_n_components= [x for x in range(2, 24, 4)]

# pca_svd_solver = ['auto', 'full', 'arpack', 'randomized']

### Setting up the Parameters with PCA Weights:

In [ ]:
params_w = {'rf__n_estimators': n_estimators_w,
          'rf__max_features': max_features_w,
          'rf__max_depth': max_depth_w,
          'rf__min_samples_split': min_samples_split_w,
          'rf__min_samples_leaf': min_samples_leaf_w,
          'rf__bootstrap': bootstrap_w}
print(params_w)

### Gridsearching the Parameters with PCA:

In [ ]:
grid = GridSearchCV(pipe_w, params_w, n_jobs=3)

### Fitting the Scaled Data with a Weighted Model:

In [ ]:
grid.fit(X_train_sc, y_train)

### Scoring the Training Data:

In [ ]:
grid.score(X_train_sc, y_train)

### Scoring the Test Data:

In [ ]:
grid.score(X_test_sc, y_test)

In [ ]:
plt.scatter(y_test, grid.predict(X_test_sc))
plt.xlabel('Actual')
plt.ylabel('Predicted')